# What modified base on Diffms approaches:
Reference files: 
1) /DiffMS/src/datasets/spec2mol_dataset.py
2) /DiffMS/src/mist/data/featurizers.py
3) /DiffMS/src/spec2mol_main.py

# SpectrumProcessor vs PeakFormula Comparison

The updated `SpectrumProcessor` class represents a significant refactoring of functionality from the `PeakFormula` class in the original code. Here are the key differences:

## Purpose and Integration
- **SpectrumProcessor**: Standalone class that processes raw spectral data directly from JSON
- **PeakFormula**: Integrated featurizer within a larger framework that works with `data.Spectra` objects

## Input Processing
- **SpectrumProcessor**: Takes raw JSON strings or dictionaries directly via `process_raw_spectrum`
- **PeakFormula**: Reads spectrum data from files in a predefined directory structure

## Architecture
- **SpectrumProcessor**: Self-contained with no inheritance
- **PeakFormula**: Inherits from `SpecFeaturizer` which inherits from `Featurizer`

## Key Implementation Differences

1. **Data Flow**:
   - `SpectrumProcessor` operates on raw data in memory
   - `PeakFormula` requires file system integration and file reading

2. **Method Structure**:
   - `SpectrumProcessor` has cleaner method names (`_get_peak_dict_from_raw`, `process_raw_spectrum`)
   - `PeakFormula` uses framework methods like `_featurize` and `collate_fn`

3. **Output Format**:
   - Both generate similar feature dictionaries but `PeakFormula` has additional metadata for the framework
   - `PeakFormula` includes collation functions for batching data in PyTorch

The `SpectrumProcessor` appears to be a focused extraction of the core spectrum processing functionality, making it more portable and independent from the larger framework. It's designed to work with raw data directly rather than relying on file system integration.

# FingerprintProcessor vs FingerprintFeaturizer Comparison

The new `FingerprintProcessor` class represents a streamlined version of the original `FingerprintFeaturizer` class with different interfaces and additional capabilities. Here's a detailed comparison:

## Purpose and Integration
- **FingerprintProcessor**: Self-contained class that processes raw spectral JSON data directly to generate fingerprints
- **FingerprintFeaturizer**: Integrated into a larger framework that works with `data.Mol` objects and includes PyTorch data handling

## Input Processing
- **FingerprintProcessor**: Accepts raw JSON with SMILES/InChI and creates molecules directly
- **FingerprintFeaturizer**: Expects pre-processed `data.Mol` objects and has no direct JSON handling

## Key Implementation Differences

1. **Data Flow**:
   - `FingerprintProcessor` extracts SMILES/InChI from JSON and creates RDKit molecules
   - `FingerprintFeaturizer` uses molecules provided by the larger framework

2. **Fingerprint Handling**:
   - Both support multiple fingerprint types (Morgan, MACCS)
   - Implementation of specific fingerprint methods is very similar

3. **New Features in FingerprintProcessor**:
   - Built-in fingerprint augmentation via bit flipping
   - Better error handling with `_empty_fingerprints` for when molecules can't be created

4. **Features in FingerprintFeaturizer not in FingerprintProcessor**:
   - Caching mechanism for fingerprints
   - Loading pre-computed fingerprints from files
   - PyTorch collation function

5. **Architecture**:
   - `FingerprintProcessor` is self-contained with no inheritance
   - `FingerprintFeaturizer` inherits from `MolFeaturizer` which inherits from `Featurizer`

The `FingerprintProcessor` represents a more independent implementation that works directly with raw data, while adding data augmentation capabilities and better error handling. However, it loses some of the framework integration features like PyTorch batching and distance calculations that were present in the original `FingerprintFeaturizer`.

# GraphProcessor vs GraphFeaturizer Comparison

The `GraphProcessor` class represents a significant enhancement and extension of the original `GraphFeaturizer` class. Here's a detailed comparison:

## Purpose and Integration
- **GraphProcessor**: Self-contained processor that converts raw spectral JSON directly into molecular graph representations
- **GraphFeaturizer**: Integrated into a larger framework that works with `data.Mol` objects from the larger system

## Input Processing
- **GraphProcessor**: Accepts raw JSON with SMILES/InChI and creates RDKit molecules directly
- **GraphFeaturizer**: Expects pre-processed `data.Mol` objects with molecule information already extracted

## Key Implementation Differences

1. **Graph Construction**:
   - Both construct PyTorch Geometric `Data` objects with similar node and edge representations
   - Similar atom type and bond type encoding approaches with one-hot encoding

2. **New Features in GraphProcessor**:
   - **Extended node features**: Option to add detailed atom properties (degree, charge, hybridization, etc.)
   - **Graph augmentation**: Ability to perturb node and edge features for data augmentation
   - **Error handling**: `_empty_graph()` method to handle cases when molecules can't be created
   - **Direct JSON processing**: Works directly with raw spectral data without requiring pre-processing

3. **Missing in GraphProcessor**:
   - **Batch collation**: `GraphFeaturizer` includes a `collate_fn` for PyTorch DataLoader integration

4. **Code Structure**:
   - `GraphProcessor` has more comprehensive and modular code organization
   - More extensive documentation and better error handling
   - More flexible input handling (SMILES or InChI)

5. **Architecture**:
   - `GraphProcessor` is self-contained with no inheritance
   - `GraphFeaturizer` inherits from `Featurizer` which enables caching behavior

The `GraphProcessor` represents a significantly enhanced version of the graph generation capabilities, with better flexibility, error handling, and feature representation capabilities. It works independently of the larger framework while adding more sophisticated graph generation and augmentation capabilities.

The major improvement is the ability to work directly with raw spectral data and to provide richer molecular representations with extended node features and augmentation options.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../../data/production_ready_data/train/spectrs/MassSpecGym_fixed.csv")

In [4]:
from processors.processors_diffms import *

/home/i_golov/.cache/pypoetry/virtualenvs/spectrum-to-molecular-ECHCqm5f-py3.10/lib/python3.10/site-packages/torch_geometric/utils/convert.py:4: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.0)
  import scipy.sparse


In [5]:
# Initialize the processor
processor = SpectrumProcessor(
    max_peaks=50,
    inten_transform="float",
    cls_type="ms1",
    magma_aux_loss=True,
    magma_folder='../../data/raw/msg_diffms/magma_outputs/magma_tsv'
)

# For fingerprints
fingerprint_processor = FingerprintProcessor(
    fp_names=["morgan2048", "maccs"],
    augment_data=False
)

# For graphs
graph_processor = GraphProcessor(
    augment_data=False,
    add_node_features=True
)
# Process each row in the DataFrame
processed_rows = []

Found 231104 MAGMA files


In [6]:
from tqdm.notebook import tqdm


for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing spectra"):
    
    # fp_features = fingerprint_processor.process_raw_spectrum(
    #     raw_spectral_json=row,
    #     spec_id=row['spec'],
    #     train_mode=False
    # )
    
    # print(fp_features)
    
    spec_features = processor.process_raw_spectrum(
        raw_spectral_json=row['extracted_spectral_info'],
        spec_id=row['spec'],
        train_mode=False
    )
    
    print(spec_features)
    
    # graph_data = graph_processor.process_raw_spectrum(
    #     raw_spectral_json=row,
    #     spec_id=row['spec'],
    #     train_mode=False
    # )
    
    # print(graph_data)

    break

Processing spectra:   0%|          | 0/231104 [00:00<?, ?it/s]

{'peak_type': array([0, 0, 0, 0, 0, 0, 0, 0, 3]), 'form_vec': array([[ 6.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         3.,  0.,  0.,  0.,  0.],
       [14., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         3.,  0.,  0.,  0.,  0.],
       [ 7.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         3.,  0.,  0.,  0.,  0.],
       [13., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.],
       [ 7.,  6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [14., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         3.,  0.,  0.,  0.,  0.],
       [ 7.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         3.,  0.,  0.,  0.,  0.],
       [13., 13.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         1.,  0.,  0.,  0.,  0.],
       [16., 17.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         

In [7]:
spec_features

{'peak_type': array([0, 0, 0, 0, 0, 0, 0, 0, 3]),
 'form_vec': array([[ 6.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          3.,  0.,  0.,  0.,  0.],
        [14., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
          3.,  0.,  0.,  0.,  0.],
        [ 7.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
          3.,  0.,  0.,  0.,  0.],
        [13., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          1.,  0.,  0.,  0.,  0.],
        [ 7.,  6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.],
        [14., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          3.,  0.,  0.,  0.,  0.],
        [ 7.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
          3.,  0.,  0.,  0.,  0.],
        [13., 13.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
          1.,  0.,  0.,  0.,  0.],
        [16., 17.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1